# Imports

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms, utils
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import h5py
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from multiprocessing import Process, Queue, Pool


# Setting up GPU (if available)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Ensuring determinism (a.k.a. seeding the PRNG)

In [3]:
random.seed(123)

# Preparing the data

## Setting up Kaggle API

In [4]:
!pip3 install kaggle --user
!mkdir /root/.kaggle
!echo '{"username":"olefredrik","key":"0e5ab392a44e3df6cb6b5659f066569d"}' > /root/.kaggle/kaggle.json

## Downloading dataset

In [5]:
!kaggle datasets download awsaf49/brats2020-training-data

100% 6.75G/6.76G [02:32<00:00, 54.9MB/s]
100% 6.76G/6.76G [02:32<00:00, 47.7MB/s]


## Unpacking dataset

In [6]:
!unzip brats2020-training-data.zip

Streaming output truncated to the last 5000 lines.
  inflating: BraTS2020_training_data/content/data/volume_70_slice_63.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_64.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_65.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_66.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_67.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_68.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_69.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_7.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_70.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_71.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_72.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_slice_73.h5  
  inflating: BraTS2020_training_data/content/data/volume_70_sl

## Setting up DataLoader

In [7]:
class BrainTumorDataset(Dataset):

    def __init__(self, csv_file, root_dir, indices=None, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            indices (list, optional): List with integers with the index of the people included
                (to seperate people in the training and test set)
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.brain_frame = pd.read_csv(csv_file)
        if indices is not None:
            l = [i for i, t in enumerate(self.brain_frame["slice_path"].to_numpy()) if int(t[t.index("volume_")+7 : t.index("_slice")]) in indices]
            print(len(l))
            self.brain_frame = self.brain_frame.iloc[l, :]
        self.brain_frame = self.brain_frame.sample(frac=1).reset_index(drop=True)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.brain_frame)
    
    def get_label(self, idx):
        return self.brain_frame["target"][idx]
    
    def push_item(self, q, idx):
        q.put(self.__getitem__(idx))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if type(idx) == slice:
            return np.array([self.__getitem__(i)[0] for i in range(len(self))[idx]]), [self.get_label(i) for i in range(len(self))[idx]]
        
        img_path = self.brain_frame["slice_path"][idx]
        img_path = img_path[img_path.index("Bra"):]
        img_path = os.path.join(self.root_dir, img_path)

        with h5py.File(img_path,'r') as hf5:
            sample = hf5["image"][:]

        if np.isnan(sample).any():
            print(f"nan i {ind}")
        sample = (sample - np.min(sample))/max(0.001, (np.max(sample) - np.min(sample)))
        if self.transform:
            sample = self.transform(sample)

        return sample, self.brain_frame["target"][idx]

## Creating PCA for dimensionality reduction (the input here is four-dimensional, while the input dimension of the network is three-dimensional)

### Choosing input data for PCA

In [8]:
tumor_data = BrainTumorDataset("BraTS20 Training Metadata.csv", "./")
X = np.array([tumor_data[i][0] for i in random.choices(range(len(tumor_data)), k=400)])
X.resize((240*240*400, 4))

### Setting up PCA

In [9]:
pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=3))])

### Fitting PCA

In [10]:
pipeline.fit(X)

Pipeline(memory=None,
         steps=[('scaling',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False))],
         verbose=False)

## Setting up transformation

In [11]:
transformation = lambda x: np.resize(transform.resize(np.resize(pipeline.transform(np.resize(x, (240*240, 4))), (240, 240, 4)), (224, 224)), (3, 224, 224))

## Creating train-test-split

In [12]:
people = set(range(1, 370))
test = set(random.sample(people, int(0.25*len(people))))
train = people - test
train = list(train)
test = list(test)
print(f"len(train): {len(train)}, len(test): {len(test)}") 

len(train): 277, len(test): 92


## Creating train and test dataloaders

**OBS:** Uses small subset of the data for testing purposes

In [13]:
train_dataloader = DataLoader(BrainTumorDataset("BraTS20 Training Metadata.csv", "./", [0,1,2,3,4,5,6,7,8,9,10,11,12,13], transformation),
           batch_size=32, shuffle=True, sampler=None,
           batch_sampler=None, num_workers=8, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, persistent_workers=False)
test_dataloader = DataLoader(BrainTumorDataset("BraTS20 Training Metadata.csv", "./", [14,15,16,17], transformation),
           batch_size=32, shuffle=True, sampler=None,
           batch_sampler=None, num_workers=8, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, persistent_workers=False)

2015
620


# Setting up model

## Training function

In [14]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_examples = 0
            # Iterate over data.
            time0 = time.time()
            for inputs, labels in dataloaders[phase]:
                print("Time loading:", time.time() - time0)
                time0 = time.time()
                print("New batch")
                inputs = inputs.float()
                inputs = inputs.to(device)
                labels = labels.float()
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    labels = labels.reshape_as(outputs)
                    preds = torch.round(torch.sigmoid(outputs))
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)
                running_examples += len(inputs)
                
                print("Time executing:", time.time() - time0)
                time0 = time.time()
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / running_examples
            epoch_acc = running_corrects.double() / running_examples

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Creating model

In [15]:
model_conv = models.vgg16(pretrained=False)
for param in model_conv.parameters():
    param.requires_grad = True

# Parameters of newly constructed modules have requires_grad=True by default
model_conv.classifier[6] = nn.Linear(4096, 1)

model_conv = model_conv.to(device)

criterion = nn.BCEWithLogitsLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.classifier[6].parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# Training model

In [16]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, {"train": train_dataloader, "val": test_dataloader}, num_epochs=25)

Streaming output truncated to the last 5000 lines.
Time executing: 0.5946071147918701
Time loading: 9.799003601074219e-05
New batch
Time executing: 0.589111328125
Time loading: 0.0017902851104736328
New batch
Time executing: 0.60103440284729
Time loading: 9.989738464355469e-05
New batch
Time executing: 0.5819823741912842
Time loading: 0.013799667358398438
New batch
Time executing: 0.5707235336303711
Time loading: 0.015474796295166016
New batch
Time executing: 0.5730311870574951
Time loading: 3.840813398361206
New batch
Time executing: 0.6010637283325195
Time loading: 9.179115295410156e-05
New batch
Time executing: 0.6075975894927979
Time loading: 0.07318615913391113
New batch
Time executing: 0.5660896301269531
Time loading: 9.059906005859375e-05
New batch
Time executing: 0.5923092365264893
Time loading: 0.00010228157043457031
New batch
Time executing: 0.6014113426208496
Time loading: 0.00025153160095214844
New batch
Time executing: 0.6084542274475098
Time loading: 0.012332916259765625


# Sandbox

In [17]:
import multiprocessing

multiprocessing.cpu_count()

2

In [18]:
batch_size = 32
#[test_dataloader[i:i+batch_size] for i in range(0, len(test_dataloader), batch_size)])
np.array(test_dataloader[0:10]).shape

TypeError: ignored